In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt

%matplotlib inline

## Confiugre IDseq Reports to match format of other data

1. Download the IDseq reports files from the UI for project = Cell Reports Benchmarking Metagenomic Tools
2. run this script to reformat the results into the format for the Cell paper benchmarking notebook.


In [2]:
idseq_reports_dir = './data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/'

filename_list = glob.glob(idseq_reports_dir + '*.csv')
sample_names = [i.split('/')[-1].split('.')[0].replace('-','.') for i in filename_list]
sample_names

['UnAmbiguouslyMapped_ds.soil',
 'UnAmbiguouslyMapped_ds.buccal',
 'UnAmbiguouslyMapped_ds.cityparks',
 'UnAmbiguouslyMapped_ds.gut',
 'UnAmbiguouslyMapped_ds.hous1',
 'UnAmbiguouslyMapped_ds.7',
 'UnAmbiguouslyMapped_ds.hous2',
 'UnAmbiguouslyMapped_ds.nycsm',
 'atcc_even',
 'atcc_staggered']

In [3]:
database = 'default'
ntnr_dict = {'nt':'NT_r', 'nr':'NR_r'}

list_of_result_dfs = []

for filename in filename_list:
    
    print(filename)
    
    sample_name = filename.split('/')[-1].split('.')[0].replace('-','.')
    print(sample_name)
    
    file_df = pd.read_csv(filename)
    
    for j in ['nt','nr', 'ntnr_conc', 'ntnr_conc_stringent']:
        classifier = 'idseq_' + j
        
        metric = j
        if j in ['ntnr_conc','ntnr_conc_stringent']: # for concordance, use NT values when NR r > 0
            metric = 'nt'
        
        # select only the species-level taxa
        species = file_df[file_df.tax_level == 1]
        
        # keep species with r > 0
        species = species[species[ntnr_dict[metric]] > 0]
        
        if j == 'ntnr_conc': # require NT and NR be greater than zero
            species = species[species['NT_r'] > 0]
            species = species[species['NR_r'] > 0]
            
        if j == 'ntnr_conc_stringent':
            species = species[species['NT_rpm'] > 10]
            species = species[species['NR_rpm'] > 10]
            
        # take a subset of the columns
        species = species[['tax_id','name',ntnr_dict[metric]]]
        
        # convert raw r values to proportion of total
        species['cum_abundance'] = species[ntnr_dict[metric]] / species[ntnr_dict[metric]].sum()
        
        # append fields required by Ye et al script
        species['classrank'] = ['species' for i in range(len(species.index))]
        species['rank'] = ['species' for i in range(len(species.index))]
        species['classifier'] = [classifier for i in range(len(species.index))]
        species['database'] = [database for i in range(len(species.index))]
        species['sample'] = sample_name
        species['cls_cum_abundance'] = [None for i in range(len(species.index))]
        species['unique_abundance'] = [None for i in range(len(species.index))]
        species['cls_unique_abundance'] = [None for i in range(len(species.index))]

        species = species[['sample', 'classifier','database','tax_id','name','cum_abundance','cls_cum_abundance',
                      'unique_abundance','cls_unique_abundance','classrank','rank']]  


        # select only teh genus-level taxa
        genus = file_df[file_df.tax_level == 2]#[['tax_id','name',ntnr_dict[metric]]]
        #genus.columns = ['taxid','name',ntnr_dict[j]]
        
        # keep genera with r > 0
        genus = genus[genus[ntnr_dict[metric]] > 0]
        
        if j == 'ntnr_conc': # require NT and NR be greater than zero
            genus = genus[genus['NT_r'] > 0]
            genus = genus[genus['NR_r'] > 0]
            
        if j == 'ntnr_conc_stringent':
            genus = genus[genus['NT_rpm'] > 10]
            genus = genus[genus['NR_rpm'] > 10]
            
        # take a subset of the columns
        genus = genus[['tax_id','name',ntnr_dict[metric]]]
        
        
        # convert raw r values to proportion of total
        genus['cum_abundance'] = genus[ntnr_dict[metric]] / genus[ntnr_dict[metric]].sum()
        genus['classrank'] = ['genus' for i in range(len(genus.index))]
        genus['rank'] = ['genus' for i in range(len(genus.index))]
        genus['classifier'] = [classifier for i in range(len(genus.index))]
        genus['database'] = [database for i in range(len(genus.index))]
        genus['samplename'] = sample_name
        genus['sample'] = sample_name
        genus['cls_cum_abundance'] = [None for i in range(len(genus.index))]
        genus['unique_abundance'] = [None for i in range(len(genus.index))]
        genus['cls_unique_abundance'] = [None for i in range(len(genus.index))]

        genus = genus[['sample', 'classifier','database','tax_id','name','cum_abundance','cls_cum_abundance',
                      'unique_abundance','cls_unique_abundance','classrank','rank']]        
    
        list_of_result_dfs.append(species)
        list_of_result_dfs.append(genus)
    

./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/UnAmbiguouslyMapped_ds-soil.csv
UnAmbiguouslyMapped_ds.soil
./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/UnAmbiguouslyMapped_ds-buccal.csv
UnAmbiguouslyMapped_ds.buccal
./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/UnAmbiguouslyMapped_ds-cityparks.csv
UnAmbiguouslyMapped_ds.cityparks
./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/UnAmbiguouslyMapped_ds-gut.csv
UnAmbiguouslyMapped_ds.gut
./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/UnAmbiguouslyMapped_ds-hous1.csv
UnAmbiguouslyMapped_ds.hous1
./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/UnAmbiguouslyMapped_ds-7.csv
UnAmbiguouslyMapped_ds.7
./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/UnAmbiguouslyMapped_ds-hous2.csv
UnAmbiguouslyMapped_ds.hous2
./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_

In [4]:
# output the results to file: idseq_combined_results.tsv
final_results = pd.concat(list_of_result_dfs)
final_results.columns = ['sample','classifier','database','taxid','name','cum_abundance','cls_cum_abundance','unique_abundance','cls_unique_abundance','classrank','rank']
final_results.to_csv('./data/idseq/cell-reports-benchmarking-metagenomics-tools_reports_v3.13/idseq_combined_results.tsv', sep='\t', index=False)

In [5]:
final_results.head()

,sample,classifier,database,taxid,name,cum_abundance,cls_cum_abundance,unique_abundance,cls_unique_abundance,classrank,rank
1,UnAmbiguouslyMapped_ds.soil,idseq_nt,default,940615,Granulicella tundricola,0.019947,None,None,None,species,species
2,UnAmbiguouslyMapped_ds.soil,idseq_nt,default,940614,Granulicella mallensis,0.020603,None,None,None,species,species
9,UnAmbiguouslyMapped_ds.soil,idseq_nt,default,210,Helicobacter pylori,0.020289,None,None,None,species,species
10,UnAmbiguouslyMapped_ds.soil,idseq_nt,default,138563,Helicobacter cetorum,0.000001,None,None,None,species,species
15,UnAmbiguouslyMapped_ds.soil,idseq_nt,default,96345,Flavobacterium psychrophilum,0.019518,None,None,None,species,species


In [6]:
set(final_results['classifier'])

{'idseq_nr', 'idseq_nt', 'idseq_ntnr_conc', 'idseq_ntnr_conc_stringent'}

In [7]:
# Q. What percentage of the simulated reads are removed by the IDseq QC steps?

sample_table = pd.read_csv('./data/idseq/project-cell_reports_benchmarking_metagenomics_tools_sample-table_v3.13.csv')

sample_table.index = sample_table['sample_name']
print(sample_table['nonhost_reads'] / sample_table['total_reads'])

sample_table['host_reads'] = sample_table['total_reads'] - sample_table['nonhost_reads']
#sample_table[['host_reads','nonhost_reads']].plot(kind='bar',stacked=True)

sample_table[['total_reads','reads_after_star','reads_after_trimmomatic','reads_after_priceseq','reads_after_cdhitdup','nonhost_reads']]




sample_name
atcc_even                           0.849691
atcc_staggered                      0.837371
RL_S001                             0.825973
RM2_S001                            0.772459
RM2_S002                            0.823163
UnAmbiguouslyMapped_ds.7            0.852747
UnAmbiguouslyMapped_ds.buccal       0.958213
UnAmbiguouslyMapped_ds.cityparks    0.985357
UnAmbiguouslyMapped_ds.gut          0.973906
UnAmbiguouslyMapped_ds.hous1        0.980988
UnAmbiguouslyMapped_ds.hous2        0.979932
UnAmbiguouslyMapped_ds.nycsm        0.977828
UnAmbiguouslyMapped_ds.soil         0.983160
dtype: float64


,total_reads,reads_after_star,reads_after_trimmomatic,reads_after_priceseq,reads_after_cdhitdup,nonhost_reads
sample_name,,,,,,
atcc_even,4000000,3999998,3999988,3999904,3999770,3398764
atcc_staggered,4000000,3999998,3999972,3999972,3997396,3349485
RL_S001,99796358,99787104,99787100,99437858,99405860,82429052
RM2_S001,99837678,99812266,99812230,99464896,99453182,77120530
RM2_S002,99787568,99775116,99775008,99425696,99412402,82141477
UnAmbiguouslyMapped_ds.7,5727654,5727422,5727422,5727368,4894682,4884237
UnAmbiguouslyMapped_ds.buccal,600000,599955,599955,599950,575314,574928
UnAmbiguouslyMapped_ds.cityparks,1200000,1199969,1199969,1199956,1186747,1182428
UnAmbiguouslyMapped_ds.gut,500000,499994,499994,499993,487733,486953
